In [1]:
from datasets import load_dataset
from promptsource.templates import DatasetTemplates

# Load all datasets from HuggingFace `datasets`

In [2]:
# Has to be loaded manually
# load_dataset('story_cloze', data_dir='ASDF')

In [3]:
normal_dataset_names = [['ag_news'],['amazon_polarity'],['dbpedia_14'], ['imdb'], ['piqa']]
super_glue_dataset_names = [['super_glue', 'boolq'],['super_glue', 'copa'], ['super_glue', 'rte']]
glue_dataset_names =[['glue', 'qnli']]

In [ ]:
normal_dataset = {name[0]: load_dataset(*name) for name in normal_dataset_names}
super_glue_dataset = {name[1]: load_dataset(*name) for name in super_glue_dataset_names}
glue_dataset = {name[1]: load_dataset(*name) for name in glue_dataset_names}

In [5]:
all_datasets = {**normal_dataset, **super_glue_dataset, **glue_dataset}

len(all_datasets)

9

In [6]:
for dataset_name, data_splits in all_datasets.items():
    print(dataset_name, data_splits.keys())

ag_news dict_keys(['train', 'test'])
amazon_polarity dict_keys(['train', 'test'])
dbpedia_14 dict_keys(['train', 'test'])
imdb dict_keys(['train', 'test', 'unsupervised'])
piqa dict_keys(['train', 'test', 'validation'])
boolq dict_keys(['train', 'validation', 'test'])
copa dict_keys(['train', 'validation', 'test'])
rte dict_keys(['train', 'validation', 'test'])
qnli dict_keys(['train', 'validation', 'test'])


# Load Promptsource `DatasetTemplates` for each Dataset 

In [7]:
normal_dataset_to_dataset_template = {dataset_name: DatasetTemplates(dataset_name) for dataset_name in normal_dataset}
super_glue_dataset_to_dataset_template = {dataset_name[1]: DatasetTemplates(f"{dataset_name[0]}/{dataset_name[1]}") for dataset_name in super_glue_dataset_names}
glue_dataset_to_dataset_template = {dataset_name[1]: DatasetTemplates(f"{dataset_name[0]}/{dataset_name[1]}") for dataset_name in glue_dataset_names}

dataset_to_dataset_template = {**normal_dataset_to_dataset_template, **super_glue_dataset_to_dataset_template, **glue_dataset_to_dataset_template}

dataset_to_dataset_template

{'ag_news': <promptsource.templates.DatasetTemplates at 0x16e047eb0>,
 'amazon_polarity': <promptsource.templates.DatasetTemplates at 0x106b1d880>,
 'dbpedia_14': <promptsource.templates.DatasetTemplates at 0x16e047df0>,
 'imdb': <promptsource.templates.DatasetTemplates at 0x106b1dc70>,
 'piqa': <promptsource.templates.DatasetTemplates at 0x16d6a3ac0>,
 'boolq': <promptsource.templates.DatasetTemplates at 0x12e2e9820>,
 'copa': <promptsource.templates.DatasetTemplates at 0x12e2d3fd0>,
 'rte': <promptsource.templates.DatasetTemplates at 0x12e2d3f40>,
 'qnli': <promptsource.templates.DatasetTemplates at 0x16e0552b0>}

# Load all prompt template names for each dataset

In [8]:
normal_dataset_to_template_names = {dataset_name: DatasetTemplates(dataset_name).all_template_names for dataset_name in normal_dataset}
super_glue_dataset_to_template_names = {dataset_name[1]: DatasetTemplates(f"{dataset_name[0]}/{dataset_name[1]}").all_template_names for dataset_name in super_glue_dataset_names}
glue_dataset_to_template_names = {dataset_name[1]: DatasetTemplates(f"{dataset_name[0]}/{dataset_name[1]}").all_template_names for dataset_name in glue_dataset_names}

dataset_to_template_names = {**normal_dataset_to_template_names, **super_glue_dataset_to_template_names, **glue_dataset_to_template_names}

total_prompts = 0
for dataset, prompt_name in dataset_to_template_names.items():
    print(f"{len(prompt_name)} prompts found for {dataset}")
    total_prompts +=len(prompt_name)

print(f"\nFound {total_prompts} total prompts for {len(dataset_to_template_names)} datasets")
print(f"\nTotal amount of prompts used in the paper: {9 + 11 + 5 + 11 + 8 + 11 + 10+ 10 + 8 + 13 }")

7 prompts found for ag_news
9 prompts found for amazon_polarity
4 prompts found for dbpedia_14
11 prompts found for imdb
11 prompts found for piqa
10 prompts found for boolq
12 prompts found for copa
10 prompts found for rte
5 prompts found for qnli

Found 79 total prompts for 9 datasets

Total amount of prompts used in the paper: 96


In [15]:
for dataset, prompt_names in dataset_to_template_names.items():
    print('Prompt Names found for:', dataset)
    print(f"{prompt_names}\n")

Prompt Names found for: ag_news
['classify', 'classify_question_first', 'classify_with_choices', 'classify_with_choices_question_first', 'recommend', 'which_section', 'which_section_choices']

Prompt Names found for: amazon_polarity
['Is_this_product_review_positive', 'Is_this_review', 'Is_this_review_negative', 'User_recommend_this_product', 'convey_negative_or_positive_sentiment', 'flattering_or_not', 'negative_or_positive_tone', 'user_satisfied', 'would_you_buy']

Prompt Names found for: dbpedia_14
['given_a_choice_of_categories ', 'given_a_list_of_category_what_does_the_title_belong_to', 'given_list_what_category_does_the_paragraph_belong_to', 'pick_one_category_for_the_following_text']

Prompt Names found for: imdb
['Movie Expressed Sentiment', 'Movie Expressed Sentiment 2', 'Negation template for positive and negative', 'Reviewer Enjoyment', 'Reviewer Enjoyment Yes No', 'Reviewer Expressed Sentiment', 'Reviewer Opinion bad good choices', 'Reviewer Sentiment Feeling', 'Sentiment w

# Let's test a prompt on an example from `ag_news` 

In [17]:
example = all_datasets["ag_news"]["train"][1]
example

{'text': 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.',
 'label': 2}

In [18]:
dataset_to_template_names['ag_news']

['classify',
 'classify_question_first',
 'classify_with_choices',
 'classify_with_choices_question_first',
 'recommend',
 'which_section',
 'which_section_choices']

In [23]:
ag_news_template = dataset_to_dataset_template['ag_news']

for prompt_idx in dataset_to_template_names['ag_news']:
    prompt = ag_news_template[prompt_idx]
    print(prompt.apply(example))
    break

['Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market. \nWhat label best describes this news article?', 'Business']
